# Monitoramento de Notícias sobre Inflação de Alimentos

Este notebook monitora feeds RSS em busca de notícias relevantes sobre inflação de alimentos, utilizando palavras-chave específicas. O monitoramento é realizado em intervalos regulares.

---

## Importação de Bibliotecas

Nesta célula, importamos as bibliotecas necessárias para o monitoramento.

In [29]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime

In [30]:
import re

def search_keywords_in_rss(feed_urls, keywords):
    matches = []
    
    headers = {'User-Agent': 'Mozilla/5.0 (compatible; RSSBot/1.0)'}

    # Pré-compila expressões regulares para cada keyword
    keyword_patterns = {
        keyword: re.compile(rf'\b{re.escape(keyword.lower())}\b', re.IGNORECASE)
        for keyword in keywords
    }

    for url in feed_urls:
        print(f"Analisando feed: {url}")
        try:
            response = requests.get(url, headers=headers, timeout=10)

            if response.status_code == 200:
                try:
                    root = ET.fromstring(response.content)
                except ET.ParseError as e:
                    print(f"Erro de parse XML no feed {url}: {e}")
                    continue

                items = root.findall('.//item')
                print(f"Foram encontrados {len(items)} itens no feed.")

                for item in items:
                    title = item.find('title').text if item.find('title') is not None else ""
                    description = item.find('description').text if item.find('description') is not None else ""
                    link = item.find('link').text if item.find('link') is not None else ""
                    pub_date = item.find('pubDate').text if item.find('pubDate') is not None else ""

                    content = f"{title} {description}".lower()

                    for keyword, pattern in keyword_patterns.items():
                        if pattern.search(content):
                            matches.append({
                                'title': title,
                                'description': description,
                                'link': link,
                                'pub_date': pub_date,
                                'feed_url': url,
                                'matched_keyword': keyword
                            })
                            break  # evita duplicação

            else:
                print(f"Falha ao acessar feed: {url}, status: {response.status_code}")

        except requests.RequestException as e:
            print(f"Erro ao tentar acessar {url}: {e}")

    return matches


In [31]:
rss_feed_url = [
    # Feeds funcionando corretamente:
    'https://g1.globo.com/rss/g1/',  # Portal de notícias da Globo, ampla cobertura nacional, política, economia e cotidiano.
    'https://feeds.folha.uol.com.br/emcimadahora/rss091.xml',  # Folha de S.Paulo, jornal com ampla cobertura em política, economia e sociedade.
    'https://www.bbc.com/portuguese/index.xml',  # Versão em português da BBC, com ampla cobertura internacional e local.
    'https://exame.com/feed/',  # Revista Exame, especializada em economia, negócios e investimentos.
    'https://www.cartacapital.com.br/feed/',  # Carta Capital, revista focada em política, economia e sociedade.
    'https://www.istoedinheiro.com.br/feed/',  # Revista especializada em economia e negócios.
    'https://www.infomoney.com.br/feed/',  # Site especializado em economia, mercado financeiro e investimentos.
    'https://www.jovempan.com.br/feed',  # Portal da Rádio Jovem Pan, com cobertura em tempo real sobre política, economia e cotidiano.
    'https://economia.ig.com.br/rss.xml',  # Portal iG, notícias gerais com foco em economia e negócios.

    # # Feeds sem itens (provável sitemap ou estrutura não compatível):
    # 'https://www.cnnbrasil.com.br/sitemap-news.xml',  # CNN Brasil, notícias rápidas e cobertura internacional.
    # 'https://oglobo.globo.com/rss.xml',  # Jornal O Globo, notícias nacionais, internacionais, política e economia.

    # # Feeds com erro de parsing (XML inválido):
    # 'https://rss.uol.com.br/feed/noticias.xml',  # Portal UOL, um dos maiores portais com notícias gerais e economia.
    # 'https://www.terra.com.br/rss/Controller?channelid=3d5d59942b25e410VgnVCM10000098cceb0aRCRD&ctName=atomo-noticia',  # Portal Terra, notícias gerais, economia e cotidiano.

    # # Feeds com status HTTP 404 (não encontrados):
    # 'https://www.estadao.com.br/rss/ultimas.xml',  # O Estado de S. Paulo, tradicional jornal brasileiro focado em política e economia.
    # 'https://noticias.r7.com/feed.xml',  # Portal da Record com notícias gerais, economia e política.
    # 'https://brasil.elpais.com/rss/brasil/portada.xml',  # Versão brasileira do jornal espanhol El País, com foco em análise política.
    # 'https://veja.abril.com.br/rss.xml',  # Revista Veja, foco em política, economia e atualidades.
    # 'https://epocanegocios.globo.com/rss/ultimas/feed.xml',  # Época Negócios, revista de negócios e economia.
    # 'https://www.correiobraziliense.com.br/rss/noticia-brasil.xml',  # Correio Braziliense, jornal tradicional de Brasília com foco político e econômico.

    # # Feeds com falha de conexão:
    # 'https://www.valor.globo.com/rss',  # Valor Econômico, especializado em economia, mercados financeiros e negócios.
]
print(rss_feed_url)

['https://g1.globo.com/rss/g1/', 'https://feeds.folha.uol.com.br/emcimadahora/rss091.xml', 'https://www.bbc.com/portuguese/index.xml', 'https://exame.com/feed/', 'https://www.cartacapital.com.br/feed/', 'https://www.istoedinheiro.com.br/feed/', 'https://www.infomoney.com.br/feed/', 'https://www.jovempan.com.br/feed', 'https://economia.ig.com.br/rss.xml']


In [32]:
import pandas as pd

# Função para ler arquivo CSV e criar lista de keywords adicionais
def load_keywords_from_csv(csv_filepath):
    df = pd.read_csv(csv_filepath)

    # Removendo duplicatas e NaNs, convertendo para uma única lista
    keywords_adicionais = pd.unique(
        df[['categoria_grupo', 'categoria_subgrupo', 'categoria_item', 'categoria_subitem']].values.ravel()
    )

    # Removendo possíveis valores NaN e convertendo para lista
    keywords_adicionais = [str(keyword).strip() for keyword in keywords_adicionais if pd.notna(keyword)]

    return keywords_adicionais

# Exemplo de uso
csv_filepath = 'data/keywords/ipca_alimentacao_bebidas.csv'
keywords_adicionais = load_keywords_from_csv(csv_filepath)

# Keywords já existentes
keywords = ['inflação', 'preço dos alimentos', 'alta dos preços', 'IPCA', 'alimentação', 'bebidas']

# Combinando as listas de keywords
keywords.extend(keywords_adicionais)

print(keywords)


['inflação', 'preço dos alimentos', 'alta dos preços', 'IPCA', 'alimentação', 'bebidas', 'Alimentação e bebidas', 'Alimentação fora do domicílio', 'Café da manhã', 'Cafezinho', 'Cerveja', 'Chopp', 'Doces', 'Lanche', 'Outras bebidas alcoólicas', 'Refeição', 'Refrigerante e água mineral', 'Sorvete', 'Vinho', 'Alimentação no domicílio', 'Açúcares e derivados', 'Açúcar cristal', 'Açúcar demerara', 'Açúcar refinado', 'Balas', 'Chocolate e achocolatado em pó', 'Chocolate em barra e bombom', 'Doce de frutas em pasta', 'Doce de leite', 'Gelatina', 'Aves e ovos', 'Frango em pedaços', 'Frango inteiro', 'Ovo de galinha', 'Bebidas e infusões', 'Açaí (emulsão)', 'Café moído', 'Café solúvel', 'Chá', 'Chá mate (erva mate)', 'Polpa de açaí', 'Polpa de fruta (congelada)', 'Suco de frutas', 'Suco em pó', 'Carnes', 'Acém', 'Alcatra', 'Capa de filé', 'Carne de carneiro', 'Carne de porco', 'Chã de dentro', 'Contrafilé', 'Costela', 'Cupim', 'Fígado', 'Filé-mignon', 'Lagarto comum', 'Lagarto plano', 'Lagarto

In [33]:
# Executando a busca
resultados = search_keywords_in_rss(rss_feed_url, keywords)

Analisando feed: https://g1.globo.com/rss/g1/
Foram encontrados 100 itens no feed.
Analisando feed: https://feeds.folha.uol.com.br/emcimadahora/rss091.xml
Foram encontrados 100 itens no feed.
Analisando feed: https://www.bbc.com/portuguese/index.xml
Foram encontrados 19 itens no feed.
Analisando feed: https://exame.com/feed/
Foram encontrados 25 itens no feed.
Analisando feed: https://www.cartacapital.com.br/feed/
Foram encontrados 20 itens no feed.
Analisando feed: https://www.istoedinheiro.com.br/feed/
Foram encontrados 100 itens no feed.
Analisando feed: https://www.infomoney.com.br/feed/
Foram encontrados 10 itens no feed.
Analisando feed: https://www.jovempan.com.br/feed
Foram encontrados 80 itens no feed.
Analisando feed: https://economia.ig.com.br/rss.xml
Foram encontrados 51 itens no feed.


In [34]:
# Exibindo resultados
if resultados:
    print(f"Encontrados {len(resultados)} resultados:")
    for res in resultados:
        print(f"\nTítulo: {res['title']}\nLink: {res['link']}\nFonte: {res['feed_url']}\nData de publicação: {res['pub_date']}\nPalavra-chave: {res['matched_keyword']}")
else:
    print("Nenhuma notícia encontrada com as palavras-chave fornecidas.")


Encontrados 20 resultados:

Título: Aneel estabelece tarifa de Itaipu e sinaliza redução do bônus repassado ao consumidor
Link: https://g1.globo.com/economia/noticia/2025/03/25/aneel-estabelece-tarifa-de-itaipu-e-sinaliza-reducao-do-bonus-repassado-ao-consumidor.ghtml
Fonte: https://g1.globo.com/rss/g1/
Data de publicação: Tue, 25 Mar 2025 19:03:06 -0000
Palavra-chave: inflação

Título: Após STF determinar reavaliação, Justiça do Rio mantém prisão de Monique Medeiros, ré pela morte do filho, Henry Borel
Link: https://g1.globo.com/rj/rio-de-janeiro/noticia/2025/03/25/stf-reavaliacao-justica-do-rio-mantem-prisao-de-monique-medeiros.ghtml
Fonte: https://g1.globo.com/rss/g1/
Data de publicação: Tue, 25 Mar 2025 18:40:57 -0000
Palavra-chave: Fígado

Título: Sete ficam feridos em acidente com ônibus no perímetro urbano de Ipatinga
Link: https://g1.globo.com/mg/vales-mg/noticia/2025/03/25/sete-ficam-feridos-em-acidente-com-onibus-no-perimetro-urbano-de-ipatinga.ghtml
Fonte: https://g1.globo.c